In [ ]:
#Working on applying correlations to the data

#Importing the necessary libraries
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from prophet import Prophet
import scipy.stats as stats


In [ ]:
#Read in data

df_utahpop = pd.read_csv('../Resources/utah population 2000-2022 neilsberg.csv')
df_rmhi = pd.read_csv('../Resources/Real Median Household Income.csv')
df_ushpi = pd.read_csv('../Resources/Utah State House Price Index.csv')
permits_df = pd.read_csv('../Pop vs Build Permits/Resources/permits.csv')


    

In [3]:
permits_df

,DATE,UTBP1FH
0,2000-01-01,944.0
1,2000-02-01,1082.0
2,2000-03-01,1312.0
3,2000-04-01,1343.0
4,2000-05-01,1363.0
...,...,...
283,2023-08-01,1582.0
284,2023-09-01,1211.0
285,2023-10-01,1258.0
286,2023-11-01,1279.0
